In [ ]:
import tensorflow as tf
import matplotlib.pylab as plt
import numpy as np

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Defining globals

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
EPOCHS = 30
class_names = ["chihuahua", "muffin"]

In [ ]:
def predict_image(model, name, url):
    path = tf.keras.utils.get_file(name, origin=url)

    img = tf.keras.utils.load_img(
        path, target_size=(IMG_HEIGHT, IMG_WIDTH)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print("{} most likely belongs to {} with a {:.2f} percent confidence."
          .format(name, class_names[np.argmax(score)], 100 * np.max(score)))

def test_prediction(model):
    predict_image(model, "Red_Sunflower", "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg")
    predict_image(model, "Chihuahua", "https://www.purina.at/sites/default/files/styles/ttt_image_510/public/2021-02/BREED%20Hero%20Mobile_0140_chihuahua_smooth.jpg")
    predict_image(model, "Chocolate_Muffin", "https://img.chefkoch-cdn.de/rezepte/1692201277528566/bilder/1502355/crop-960x540/die-schnellsten-und-besten-muffins-ueberhaupt.jpg")
    predict_image(model, "Blueberry_Muffin", "https://upload.wikimedia.org/wikipedia/commons/8/8a/Muffin_NIH.jpg")

## Training-Test Split

In [ ]:
train_dir = "./data"
batch_size = 32

training_data = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  labels='inferred',  
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=batch_size)

validation_data = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  labels='inferred',  
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=batch_size)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

training_data = training_data.cache().shuffle(6000).prefetch(buffer_size=AUTOTUNE)
validation_data = validation_data.cache().prefetch(buffer_size=AUTOTUNE)

## Running a pretrained model

In [ ]:
base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False,
        pooling='max',
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        weights='imagenet')

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(512, 'relu'),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(2, 'softmax')
])

base_model.trainable = False
model.summary()

### Preprocessing

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

In [ ]:
data_preprocessing = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Resizing(IMG_HEIGHT, IMG_WIDTH),
])

In [ ]:
model = tf.keras.Sequential([
  data_preprocessing,
  data_augmentation,
  #tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  #tf.keras.layers.MaxPooling2D(),
  model
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
  training_data,
  validation_data=validation_data,
  epochs=EPOCHS
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_prediction(model)

## Experiment: Training a custom model yields better results

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    tf.keras.layers.Convolution2D(filters=16, kernel_size=(5,5)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    tf.keras.layers.Convolution2D(filters=32, kernel_size=(5,5)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    tf.keras.layers.Convolution2D(filters=128, kernel_size=(3,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    tf.keras.layers.Convolution2D(filters=256, kernel_size=(3,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    tf.keras.layers.Flatten(),
            
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()

In [ ]:
model = tf.keras.Sequential([
  data_preprocessing,
  data_augmentation,
  #tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  #tf.keras.layers.MaxPooling2D(),
  model
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
  training_data,
  validation_data=validation_data,
  epochs=EPOCHS
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_prediction(model)

## Results

The pretrained model detects with higher accuracy.